In [15]:
import graphlab as gl

sf = gl.SFrame({'srch_destination_id':[1, 1, 1, 1, 2, 2, 2], 
                'relevance':[4.95, 3.9, 0.05, 1.1, 0.2, 1.3, 0.05], 
                'hotel_cluster':[30, 63, 60, 25, 48, 82, 58]})

sf

hotel_cluster,relevance,srch_destination_id
30,4.95,1
63,3.9,1
60,0.05,1
25,1.1,1
48,0.2,2
82,1.3,2
58,0.05,2


In [16]:
summary = sf.groupby('srch_destination_id',{'hotel_cluster': gl.aggregate.CONCAT('relevance', 'hotel_cluster')})
summary

srch_destination_id,hotel_cluster
2,"{1.3: 82, 0.2: 48, 0.05:58} ..."
1,"{4.95: 30, 3.9: 63, 1.1:25, 0.05: 60} ..."


In [17]:
def arg_max(d, k=3):
    topk = sorted(d.keys(), reverse=True)[:k]
    return [d[k] for k in topk]

In [18]:
summary['hotel_cluster'] = summary['hotel_cluster'].apply(lambda d: arg_max(d, k=5))
summary

srch_destination_id,hotel_cluster
2,"[82.0, 48.0, 58.0]"
1,"[30.0, 63.0, 25.0, 60.0]"
